<a href="https://colab.research.google.com/github/dlwjddn29/trailing-algo-strat1/blob/main/volatility_filter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import yfinance as yf
import numpy as np
import pandas as pd

drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab/data_files/market_cap_data.csv') #read csv file to load
filtered_df = df.iloc[99:120].copy() #mid-cap performing stocks
symbols = filtered_df["Symbol"].tolist()

data = yf.download(symbols, period="6mo", auto_adjust=False)
if isinstance(data.columns, pd.MultiIndex):
    adj_close = data["Adj Close"]
else:
    adj_close = data[["Adj Close"]]

# daily_returns -> 6 month volatility
returns = adj_close.pct_change()
vol_6m = returns.std()

filtered_df["Volatility6M"] = filtered_df["Symbol"].map(vol_6m)

#further filter out low volatility stocks
'''to test out high volatility stocks for better swing trades'''
threshold = filtered_df["Volatility6M"].quantile(0.5)
low_vol_df = filtered_df[filtered_df["Volatility6M"] <= threshold].copy()

[*********************100%***********************]  21 of 21 completed


In [ ]:
low_vol_df

,Symbol,MarketCap,Sector,CompanyName,Volatility6M
101,PH,106076323840,Industrials,Parker-Hannifin Corporation,0.014421
102,MCK,106069155840,Healthcare,McKesson Corporation,0.014576
103,HCA,104809455616,Healthcare,"HCA Healthcare, Inc.",0.014018
105,ADP,102754246656,Technology,"Automatic Data Processing, Inc.",0.011298
106,CME,99987103744,Financial Services,CME Group Inc.,0.010206
107,SO,99936280576,Utilities,Southern Company (The),0.008512
108,CMCSA,98822422528,Communication Services,Comcast Corporation,0.013524
110,MO,96607551488,Consumer Defensive,"Altria Group, Inc.",0.012724
114,TT,96123863040,Industrials,Trane Technologies plc,0.014565
116,DUK,95597895680,Utilities,Duke Energy Corporation (Holdin,0.008133


In [ ]:

# --- Parameters ---
initial_capital = 100_000  # total portfolio size

# --- Step 1: Risk-adjusted weights (inverse volatility) ---
low_vol_df["Weight"] = 1 / low_vol_df["Volatility6M"]
low_vol_df["Weight"] /= low_vol_df["Weight"].sum()  # normalize sum to 1

# --- Step 2: Map current prices (from your previously downloaded adj_close) ---
# Assuming you have `adj_close` from yf.download(symbols, period="6mo", auto_adjust=False)
current_prices = adj_close.iloc[-1]  # latest Adjusted Close
low_vol_df["CurrentPrice"] = low_vol_df["Symbol"].map(current_prices)

# --- Step 3: Dollar allocation and number of shares ---
low_vol_df["DollarAllocation"] = low_vol_df["Weight"] * initial_capital
low_vol_df["SharesToBuy"] = (low_vol_df["DollarAllocation"] / low_vol_df["CurrentPrice"]).astype(int)
low_vol_df["AllocatedCapital"] = low_vol_df["SharesToBuy"] * low_vol_df["CurrentPrice"]

# --- Step 4: Calculate leftover cash ---
total_allocated = low_vol_df["AllocatedCapital"].sum()
leftover_cash = initial_capital - total_allocated

# --- Step 5: Display allocation table ---
allocation_df = low_vol_df[[
    "Symbol", "CompanyName", "CurrentPrice", "Volatility6M", "Weight",
    "DollarAllocation", "SharesToBuy", "AllocatedCapital"
]]

print(f"Total Allocated: ${total_allocated:,.2f}")
print(f"Leftover Cash: ${leftover_cash:,.2f}")
allocation_df


Total Allocated: $97,783.47
Leftover Cash: $2,216.53


,Symbol,CompanyName,CurrentPrice,Volatility6M,Weight,DollarAllocation,SharesToBuy,AllocatedCapital
101,PH,Parker-Hannifin Corporation,831.950012,0.014421,0.072849,7284.875498,8,6655.600098
102,MCK,McKesson Corporation,842.090027,0.014576,0.072073,7207.334433,8,6736.720215
103,HCA,"HCA Healthcare, Inc.",472.649994,0.014018,0.074941,7494.057748,15,7089.749908
105,ADP,"Automatic Data Processing, Inc.",253.089996,0.011298,0.092984,9298.352471,36,9111.239868
106,CME,CME Group Inc.,285.040009,0.010206,0.102929,10292.909203,36,10261.440308
107,SO,Southern Company (The),90.690002,0.008512,0.123423,12342.327604,136,12333.840332
108,CMCSA,Comcast Corporation,27.510000,0.013524,0.077676,7767.579104,282,7757.820065
110,MO,"Altria Group, Inc.",58.189999,0.012724,0.082559,8255.897699,141,8204.789806
114,TT,Trane Technologies plc,418.160004,0.014565,0.072125,7212.511641,17,7108.720062
116,DUK,Duke Energy Corporation (Holdin,122.709999,0.008133,0.129165,12916.491012,105,12884.549904


In [ ]:
allocation_df['Weight'].sum()

np.float64(1.0)